In [51]:
import pandas as pd

# 파일 읽기
file_path = 'Air_reviews.xlsx' 
df = pd.read_excel(file_path)

df

,reviews,dfake
0,"LG 공기청정기 고민했는데, 구매하고 나서 너무 다행이에요. 미세먼지가 확실히 줄어...",1
1,LG 공기청정기 구매했는데 정말 생각보다 성능이 좋네요! 미세먼지가 제거되니 공기 ...,1
2,"LG 공기청정기 구매했는데, 미세먼지 제거가 뛰어나네요. 생각보다 효과가 좋아서 매...",1
3,"LG 공기청정기 구매했는데, 생각보다 성능이 뛰어나요. 미세먼지가 확실히 줄어들고,...",1
4,"LG 공기청정기 구매했는데, 이미 사용 중인 삼성 제품과 비교해도 성능이 뛰어나네요...",1
...,...,...
7980,배송비는 바로 받아놓고 계속 전화도 안받으셨지만... 배송믄 2주 조금 더 걸렸습니...,0
7981,배송도 빠르고 친절하였습니다,0
7982,공기청정기는 lg제품이 최고입니다적당한가격에 구매하여 아껴가며 오래도록 쓰겠 습니다,0
7983,배송도 지정일에 잘왔고 싸게 잘 샀어요,0


# 1. **로지스틱 회귀 (분류)**

In [1]:

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report


In [37]:
# 데이터 섞기
df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)

# 섞인 데이터로 학습
X = df_shuffled['reviews']  # 섞인 리뷰 데이터
y = df_shuffled['dfake']     # 섞인 타겟


# 리뷰 데이터를 TF-IDF로 변환
vectorizer = TfidfVectorizer(max_features=500)  # n개의 피처(단어 중요도 높은순)를 사용
X_tfidf = vectorizer.fit_transform(X)

# 학습용 데이터와 테스트용 데이터로 나누기
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.4, random_state=42)

# 로지스틱 회귀 모델 정의 및 학습
model = LogisticRegression()
model.fit(X_train, y_train)

# 예측 수행
y_pred = model.predict(X_test)


In [39]:
# 성능 평가
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)

Accuracy: 0.9492798998121478
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.98      0.97      2375
           1       0.95      0.85      0.90       819

    accuracy                           0.95      3194
   macro avg       0.95      0.92      0.93      3194
weighted avg       0.95      0.95      0.95      3194



In [62]:
#데이터 클래스 분포
print(df['dfake'].value_counts())

dfake
0    5968
1    2017
Name: count, dtype: int64


In [33]:
#교차검증
from sklearn.model_selection import cross_val_score

model = LogisticRegression()
scores = cross_val_score(model, X_tfidf, y, cv=5)
print(f"Cross-Validation Scores: {scores}")
print(f"Average CV Score: {scores.mean()}")


Cross-Validation Scores: [0.95554164 0.96055103 0.95428929 0.95867251 0.95867251]
Average CV Score: 0.9575453976205385


## **규제(Regulation) -> 과적합 방지**
- c값이 작으면 규제 강도 높아짐 (0.01, 0.1) : 규제 강도 강해져 단순한 관계만 학습함, 과적합 방지하지만 단순한 모델이 되어 과소적합이 될 수 있음
- c값 클때(10, 100) : 더 세부적으로 학습하려고 함, 작은 패턴에 민감하게 반응해 과적합 가능성 커짐, 테스트 데이터 성능 떨어짐

In [ ]:
#규제(Regulation)사용 -> 과적합 방지
model_r = LogisticRegression(C=0.05)  # C 값을 낮추면 규제가 강해짐
model_r.fit(X_train, y_train)


# 예측 수행
y_pred_r = model_r.predict(X_test)

# 성능 평가
accuracy = accuracy_score(y_test, y_pred_r)
report = classification_report(y_test, y_pred_r)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)

### 규제 효과 실험 (c민감도)

In [58]:
for C_value in [0.01, 0.1, 1, 10, 100]:
    model_rt = LogisticRegression(C=C_value)
    model_rt.fit(X_train, y_train)
    y_pred_rt = model_rt.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred_rt)
    print(f"C = {C_value}, Accuracy = {accuracy}")

C = 0.01, Accuracy = 0.7435817157169693
C = 0.1, Accuracy = 0.8663118346900438
C = 1, Accuracy = 0.9492798998121478
C = 10, Accuracy = 0.9627426424546024
C = 100, Accuracy = 0.9643080776455855


# 2. **서포트 벡터 머신(SVM)** 

In [69]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

In [83]:
# 데이터 섞기
df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)

# 섞인 데이터로 학습
X = df_shuffled['reviews']  # 섞인 리뷰 데이터
y = df_shuffled['dfake']     # 섞인 타겟

# 리뷰 데이터를 TF-IDF로 변환
vectorizer = TfidfVectorizer(max_features=500)
X_tfidf = vectorizer.fit_transform(X)

# 학습용 데이터와 테스트용 데이터로 나누기
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.4, random_state=42)


*커널*
- 데이터를 고차원으로 변환하여 분류하는 방법 (linear, RBF 등)

In [86]:
# SVM 모델 정의 및 학습
model = SVC(kernel='linear')  # 선형 커널을 사용한 SVM
model.fit(X_train, y_train)

# 예측 수행
y_pred = model.predict(X_test)


In [88]:
# 성능 평가
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)

Accuracy: 0.960551033187226
Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.98      0.97      2375
           1       0.95      0.90      0.92       819

    accuracy                           0.96      3194
   macro avg       0.96      0.94      0.95      3194
weighted avg       0.96      0.96      0.96      3194



In [99]:
#교차 검증
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X_tfidf, y, cv=5)  # 5-fold 교차 검증
print(f"Cross-Validation Scores: {scores}")
print(f"Average CV Score: {scores.mean()}")


Cross-Validation Scores: [0.96305573 0.96931747 0.96430808 0.96180338 0.97182217]
Average CV Score: 0.9660613650594865


## **SVM에서 규제**

In [130]:
from sklearn.svm import SVC

# SVM 모델 정의 (C 값을 설정하여 규제 강도 조절)
for C_value in [0.01, 0.1, 1, 10, 100]:
    model_rt = SVC(kernel = 'linear', C=C_value)
    model_rt.fit(X_train, y_train)
    y_pred_rt = model_rt.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred_rt)
    print(f"C = {C_value}, Accuracy = {accuracy}")

C = 0.01, Accuracy = 0.9371859296482412
C = 0.1, Accuracy = 0.961892797319933
C = 1, Accuracy = 0.9752931323283082
C = 10, Accuracy = 0.9752931323283082
C = 100, Accuracy = 0.9690117252931323


## **데이터 증강(Augmentation)**
- SMOTE(Synthetic Minority Over-sampling Technique)
- 클래스 불균형이 확인된다면, 소수 클래스의 데이터를 증강하여 균형 맞춤
- 과적합 방지

In [94]:
from imblearn.over_sampling import SMOTE

# SMOTE 적용
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_tfidf, y)

# 학습용 데이터와 테스트용 데이터로 나누기
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.4, random_state=42)

# SVM 모델 학습 및 예측
model_smote = SVC(kernel='linear')
model_smote.fit(X_train, y_train)
y_pred_smote = model_smote.predict(X_test)

# 성능 평가
accuracy = accuracy_score(y_test, y_pred_smote)
report = classification_report(y_test, y_pred_smote)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)


Accuracy: 0.9719371727748691
Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.97      0.97      2364
           1       0.97      0.97      0.97      2411

    accuracy                           0.97      4775
   macro avg       0.97      0.97      0.97      4775
weighted avg       0.97      0.97      0.97      4775



# 3. **GBM(그래디언트 부스트 머신)**
- 순차적인 학습: 각 단계에서 이전 단계의 예측 오류를 보완하면서 모델을 학습
- 앙상블 기법: 여러 약한 모델을 결합해 더 강력한 모델을 만드는 방식으로, 보통 결정 트리를 약한 학습기로 사용
- 조정 가능한 하이퍼파라미터: GBM은 여러 하이퍼파라미터(예: learning_rate, n_estimators 등)를 조정해 성능을 개선

In [111]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE

In [113]:
# 데이터 섞기
df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)

# 섞인 데이터로 학습
X = df_shuffled['reviews']  # 섞인 리뷰 데이터
y = df_shuffled['dfake']     # 섞인 타겟

In [115]:
# TF-IDF 벡터화
vectorizer = TfidfVectorizer(max_features=500)
X_tfidf = vectorizer.fit_transform(X)

# 학습용 데이터와 테스트용 데이터로 나누기
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)


In [117]:
# GBM 모델 정의 및 학습
model = GradientBoostingClassifier(random_state=42)
model.fit(X_train, y_train)

# 예측 수행
y_pred = model.predict(X_test)

# 성능 평가
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)

Accuracy: 0.9298685034439574
Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.98      0.95      1191
           1       0.93      0.79      0.85       406

    accuracy                           0.93      1597
   macro avg       0.93      0.88      0.90      1597
weighted avg       0.93      0.93      0.93      1597



## **GBM에서 규제**
- **learning_rate**와 **n_estimators**가 규제 역할
- **learning_rate**: 학습률을 낮추면 각 단계에서 모델이 천천히 학습, 작은 변화를 점진적으로 반영하여 과적합 방지
- **n_estimators**: 트리의 개수를 증가시키면 더 복잡한 모델이 될 수 있지만, 학습률이 낮으면 많은 트리로도 과적합을 피할 수 있음. 반대로, 학습률이 높고 트리 개수가 많으면 과적합의 위험이 커짐.

In [139]:
from sklearn.ensemble import GradientBoostingClassifier

# 학습률과 트리 개수의 다양한 옵션을 설정
learning_rates = [0.01, 0.1, 0.2]
n_estimators_options = [50, 100, 200]

# 민감도 분석 결과를 저장할 리스트
results = []

# 반복문을 통해 다양한 learning_rate와 n_estimators에 대한 성능을 평가
for lr in learning_rates:
    for n_est in n_estimators_options:
        model = GradientBoostingClassifier(learning_rate=lr, n_estimators=n_est, random_state=42)
        model.fit(X_train, y_train)
        
        # 예측 수행
        y_pred = model.predict(X_test)
        
        # 정확도 계산
        accuracy = accuracy_score(y_test, y_pred)
        
        # 결과 저장
        results.append({
            'learning_rate': lr,
            'n_estimators': n_est,
            'accuracy': accuracy
        })

# 결과 출력
results_df = pd.DataFrame(results)
print(results_df)

   learning_rate  n_estimators  accuracy
0           0.01            50  0.833752
1           0.01           100  0.851340
2           0.01           200  0.881910
3           0.10            50  0.926298
4           0.10           100  0.945980
5           0.10           200  0.956868
6           0.20            50  0.943049
7           0.20           100  0.957286
8           0.20           200  0.969430


## **데이터 증강(Augmentation)**
- SMOTE(Synthetic Minority Over-sampling Technique)


In [125]:
# SMOTE 적용
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_tfidf, y)

# 학습용 데이터와 테스트용 데이터로 나누기
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# GBM 모델 정의 및 학습
model_smote = GradientBoostingClassifier(random_state=42)
model_smote.fit(X_train, y_train)

# 예측 수행
y_pred_smote = model_smote.predict(X_test)

# 성능 평가
accuracy = accuracy_score(y_test, y_pred_smote)
report = classification_report(y_test, y_pred_smote)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)

Accuracy: 0.9459798994974874
Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.97      0.95      1188
           1       0.97      0.92      0.94      1200

    accuracy                           0.95      2388
   macro avg       0.95      0.95      0.95      2388
weighted avg       0.95      0.95      0.95      2388

